In [10]:
from routrie import Router as RoutrieRouter
from http_router import Router as HTTPRouter
from starlette.routing import Route, Router as StarletteRouter
from starlette.requests import Request

In [11]:
routrie_router: RoutrieRouter[int] = RoutrieRouter()
routrie_router.insert("/path", 1)
routrie_router.insert("/path/users", 2)

In [12]:
%%timeit
for _ in range(10_000):
    routrie_router.find("/path")

2.4 ms ± 38.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
http_router = HTTPRouter()
http_router.route('/path')(1)
http_router.route('/path/users')(2)

2

In [14]:
%%timeit
for _ in range(10_000):
    http_router.find("/path")

4.87 ms ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
async def endpoint1(request: Request):
    assert request.url.path == "/path"

async def endpoint2(request: Request):
    assert request.url.path == "/path/users"

starlette_router = StarletteRouter(
    routes=[
        Route("/path", endpoint=endpoint1),
        Route("/path/users", endpoint=endpoint2),
    ]
)

scope1 = {
    "type": "http",
    "method": "GET",
    "path": "/path"
}

scope2 = {
    "type": "http",
    "method": "GET",
    "path": "/path/users"
}

In [16]:
%%timeit
# simulate what Starlette does internally
for _ in range(10_000):
    for route in starlette_router.routes:
        match, _ = route.matches(scope1)
        if match == match.FULL:
            break
    for route in starlette_router.routes:
        match, _ = route.matches(scope2)
        if match == match.FULL:
            break

21 ms ± 100 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
